# **1. Perkenalan Dataset**


Tahap pertama, Anda harus mencari dan menggunakan dataset dengan ketentuan sebagai berikut:

1. **Sumber Dataset**:  
   Dataset dapat diperoleh dari berbagai sumber, seperti public repositories (*Kaggle*, *UCI ML Repository*, *Open Data*) atau data primer yang Anda kumpulkan sendiri.


# **2. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning atau deep learning.

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

# Download resources jika belum ada (hanya perlu dijalankan sekali)
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\FathurNitro\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\FathurNitro\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\FathurNitro\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

# **3. Memuat Dataset**

Pada tahap ini, Anda perlu memuat dataset ke dalam notebook. Jika dataset dalam format CSV, Anda bisa menggunakan pustaka pandas untuk membacanya. Pastikan untuk mengecek beberapa baris awal dataset untuk memahami strukturnya dan memastikan data telah dimuat dengan benar.

Jika dataset berada di Google Drive, pastikan Anda menghubungkan Google Drive ke Colab terlebih dahulu. Setelah dataset berhasil dimuat, langkah berikutnya adalah memeriksa kesesuaian data dan siap untuk dianalisis lebih lanjut.

Jika dataset berupa unstructured data, silakan sesuaikan dengan format seperti kelas Machine Learning Pengembangan atau Machine Learning Terapan

In [17]:
# 1. Load data
from pathlib import Path

# Gunakan raw string (r"...") atau pathlib untuk menghindari unicode escape
file_path = Path(r"C:\Users\FathurNitro\OneDrive\Documents\sub dicoding\membangun machine learning bismillah\amazon.csv")

if not file_path.exists():
    raise FileNotFoundError(f"File not found: {file_path}")

df = pd.read_csv(file_path)

# Tampilkan kolom untuk debugging
print("Columns:", df.columns.tolist())

# Deteksi kolom teks yang mungkin ada di dataset
possible_text = ['review_text', 'review_content', 'review', 'about_product', 'review_title']
text_col = next((c for c in possible_text if c in df.columns), None)
if text_col is None:
    raise KeyError(f"No text column found. Available columns: {df.columns.tolist()}")

# Jika kolom 'sentiment' tidak ada, coba buat dari 'rating' jika tersedia
if 'sentiment' not in df.columns:
    if 'rating' in df.columns:
        df['rating_num'] = pd.to_numeric(df['rating'], errors='coerce')
        # Threshold sederhana: rating >= 4 => positive, else negative
        df['sentiment'] = df['rating_num'].apply(lambda r: 'positive' if pd.notna(r) and r >= 4 else ('negative' if pd.notna(r) else np.nan))
        print("Created 'sentiment' from 'rating' (>=4 => positive).")
    else:
        raise KeyError("No 'sentiment' or 'rating' column found. Please provide a label column or mapping.")

# Rename kolom teks ke 'review_text' agar sel-sel berikutnya tetap bekerja
if text_col != 'review_text':
    df.rename(columns={text_col: 'review_text'}, inplace=True)
    print(f"Renamed text column '{text_col}' to 'review_text'.")

# Tampilkan 5 baris pertama
print("--- 5 Baris Pertama Data ---")
print(df.head())

# Tampilkan informasi dasar data
print("\n--- Informasi Dataset ---")
df.info()

# Cek distribusi target
print("\n--- Distribusi Kolom Target ('sentiment') ---")
print(df['sentiment'].value_counts(dropna=False))

Columns: ['product_id', 'product_name', 'category', 'discounted_price', 'actual_price', 'discount_percentage', 'rating', 'rating_count', 'about_product', 'user_id', 'user_name', 'review_id', 'review_title', 'review_content', 'img_link', 'product_link']
Created 'sentiment' from 'rating' (>=4 => positive).
Renamed text column 'review_content' to 'review_text'.
--- 5 Baris Pertama Data ---
   product_id                                       product_name  \
0  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1  B098NS6PVG  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...   
2  B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
3  B08HDJ86NZ  boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...   
4  B08CF3B7N1  Portronics Konnect L 1.2M Fast Charging 3A 8 P...   

                                            category discounted_price  \
0  Computers&Accessories|Accessories&Peripherals|...             ₹399   
1  Computers&Accessories|Accessories&Peripherals|...   

# **4. Exploratory Data Analysis (EDA)**

Pada tahap ini, Anda akan melakukan **Exploratory Data Analysis (EDA)** untuk memahami karakteristik dataset.

Tujuan dari EDA adalah untuk memperoleh wawasan awal yang mendalam mengenai data dan menentukan langkah selanjutnya dalam analisis atau pemodelan.

In [18]:
# 2. EDA

# a. Cek Missing Values
print("\n--- Missing Values ---")
print(df.isnull().sum())

# b. Cek Duplikasi
print("\n--- Jumlah Data Duplikat ---")
print(f"Total Duplikat: {df.duplicated().sum()}")

# c. Statistik Deskriptif (hanya akan berguna untuk kolom numerik, tapi baik untuk ditampilkan)
print("\n--- Statistik Deskriptif ---")
print(df.describe(include='all'))

# d. Analisis Teks (Contoh: Panjang Review)
df['review_length'] = df['review_text'].apply(len)
print("\n--- Statistik Panjang Review ---")
print(df['review_length'].describe())

# Catatan EDA:
# - Tidak ada missing values yang terlihat.
# - Ada sejumlah data duplikat yang dapat dihapus.
# - Target 'sentiment' adalah binary (positive, negative).


--- Missing Values ---
product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0
rating_count           2
about_product          0
user_id                0
user_name              0
review_id              0
review_title           0
review_text            0
img_link               0
product_link           0
rating_num             1
sentiment              1
dtype: int64

--- Jumlah Data Duplikat ---
Total Duplikat: 0

--- Statistik Deskriptif ---
        product_id                                       product_name  \
count         1465                                               1465   
unique        1351                                               1337   
top     B08WRWPM22  Fire-Boltt Ninja Call Pro Plus 1.83" Smart Wat...   
freq             3                                                  5   
mean           NaN                                                NaN 

# **5. Data Preprocessing**

Pada tahap ini, data preprocessing adalah langkah penting untuk memastikan kualitas data sebelum digunakan dalam model machine learning.

Jika Anda menggunakan data teks, data mentah sering kali mengandung nilai kosong, duplikasi, atau rentang nilai yang tidak konsisten, yang dapat memengaruhi kinerja model. Oleh karena itu, proses ini bertujuan untuk membersihkan dan mempersiapkan data agar analisis berjalan optimal.

Berikut adalah tahapan-tahapan yang bisa dilakukan, tetapi **tidak terbatas** pada:
1. Menghapus atau Menangani Data Kosong (Missing Values)
2. Menghapus Data Duplikat
3. Normalisasi atau Standarisasi Fitur
4. Deteksi dan Penanganan Outlier
5. Encoding Data Kategorikal
6. Binning (Pengelompokan Data)

Cukup sesuaikan dengan karakteristik data yang kamu gunakan yah. Khususnya ketika kami menggunakan data tidak terstruktur.

In [19]:
# Hapus data duplikat
df.drop_duplicates(inplace=True)
print(f"\nJumlah data setelah hapus duplikat: {len(df)}")


Jumlah data setelah hapus duplikat: 1465


In [21]:
lemmatizer = WordNetLemmatizer()
# stop_words mungkin belum tersedia saat runtime, ambil secara dinamis di dalam fungsi jika perlu

def preprocess_text(text):
    # Tangani nilai kosong
    if pd.isna(text):
        return ""
    text = str(text).lower()

    # Hapus karakter non-alfabet (ganti dengan spasi untuk menjaga pemisahan kata)
    text = re.sub(r'[^a-z\s]', ' ', text)

    # Tokenisasi dengan fallback jika resource NLTK tidak tersedia
    try:
        words = nltk.word_tokenize(text)
    except LookupError:
        # Coba download punkt jika belum ada, lalu ulangi. Jika masih gagal, fallback ke split sederhana.
        try:
            nltk.download('punkt', quiet=True)
            words = nltk.word_tokenize(text)
        except Exception:
            words = text.split()

    # Pastikan stopwords tersedia
    try:
        stop_words = set(stopwords.words('english'))
    except LookupError:
        nltk.download('stopwords', quiet=True)
        stop_words = set(stopwords.words('english'))

    # Lemmatize dan hapus stopwords/word pendek
    cleaned = [lemmatizer.lemmatize(w) for w in words if w not in stop_words and len(w) > 1]

    return " ".join(cleaned)

# Terapkan fungsi preprocessing (isi NaN menjadi empty string agar apply aman)
df['clean_review'] = df['review_text'].fillna("").apply(preprocess_text)

# Tampilkan perbandingan
print("\n--- Perbandingan Teks Asli vs. Bersih ---")
print("Original:", df['review_text'].iloc[0])
print("Cleaned: ", df['clean_review'].iloc[0])


--- Perbandingan Teks Asli vs. Bersih ---
Original: Looks durable Charging is fine tooNo complains,Charging is really fast, good product.,Till now satisfied with the quality.,This is a good product . The charging speed is slower than the original iPhone cable,Good quality, would recommend,https://m.media-amazon.com/images/W/WEBP_402378-T1/images/I/81---F1ZgHL._SY88.jpg,Product had worked well till date and was having no issue.Cable is also sturdy enough...Have asked for replacement and company is doing the same...,Value for money
Cleaned:  look durable charging fine toono complains charging really fast good product till satisfied quality good product charging speed slower original iphone cable good quality would recommend http medium amazon com image webp image zghl sy jpg product worked well till date issue cable also sturdy enough asked replacement company value money


In [25]:
# 3. Encoding target
# Buang baris tanpa label 'sentiment'
missing_sent = df['sentiment'].isnull().sum()
if missing_sent > 0:
    print(f"Dropping {missing_sent} rows without sentiment label")
    df = df[df['sentiment'].notna()].copy()

# Standardisasi beberapa kemungkinan nilai lalu encoding
df['sentiment'] = df['sentiment'].astype(str).str.lower().str.strip()
# Jika ada varian seperti 'pos','neg','1','0' coba koersi sederhana
df['sentiment'] = df['sentiment'].replace({
    'pos': 'positive', 'p': 'positive', '1': 'positive', 'true': 'positive', 'yes': 'positive',
    'neg': 'negative', 'n': 'negative', '0': 'negative', 'false': 'negative', 'no': 'negative'
})

df['sentiment_encoded'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# Jika masih ada nilai tak terduga, buang baris tersebut
if df['sentiment_encoded'].isnull().any():
    cnt = df['sentiment_encoded'].isnull().sum()
    print(f"Dropping {cnt} rows with unknown sentiment after coercion")
    df = df[df['sentiment_encoded'].notna()].copy()

# Reset index untuk menghindari masalah indexing saat split
df = df.reset_index(drop=True)

# 4. Feature dan Target
X = df['clean_review']
y = df['sentiment_encoded'].astype(int)

# Pastikan setidaknya ada 2 kelas untuk stratify
if y.nunique() < 2:
    raise ValueError("Target memiliki kurang dari 2 kelas setelah pembersihan. Tidak dapat melakukan stratified split.")

# 5. Data Splitting
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Gabungkan kembali untuk disimpan (lebih aman dan jelas)
df_train = pd.DataFrame({'review': X_train.values, 'sentiment': y_train.values})
df_train['split'] = 'train'
df_test = pd.DataFrame({'review': X_test.values, 'sentiment': y_test.values})
df_test['split'] = 'test'
df_preprocessed = pd.concat([df_train, df_test], ignore_index=True)

# Simpan hasil preprocessing ke folder preprocessing (lokasi tetap)
from pathlib import Path
output_folder = Path(r"C:\Users\FathurNitro\OneDrive\Documents\sub dicoding\membangun machine learning bismillah\preprocessing")
output_folder.mkdir(parents=True, exist_ok=True)

output_file = output_folder / 'amazon_preprocessed.csv'
df_preprocessed.to_csv(output_file, index=False)

print(f"\nData preprocessed disimpan ke: {output_file}")


Data preprocessed disimpan ke: C:\Users\FathurNitro\OneDrive\Documents\sub dicoding\membangun machine learning bismillah\preprocessing\amazon_preprocessed.csv
